In [1]:
# Imports

import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split

## Loading in Data

Data Description from Github:
_______________________

Word Complexity Lexicon (https://github.com/mounicam/lexical_simplification/tree/master/word_complexity_lexicon)
_______________________


lexicon.tsv : Each line consists of word and its complexity scores calculated by aggregating over human ratings. 
              The score belongs to a scale of 1-6, where 1 represents "very simple" and 6 represents "very complex"

lexion_annotations.tsv: Each line consists of a word in the lexicon and its individual ratings from 11 annotators.
                        Each rating again belongs to the scale of 1-6. -1 indicates that the annotator did not rate
                        the word.
                        
NOTE: Both the files are tab delimited.

In [2]:
lexicon_data = pd.read_csv('../WCL_data/lexicon.csv')
lexicon_ann = pd.read_csv('../WCL_data/lexicon_annotations.csv')

## Data Cleaning

In [3]:
# TODO: 
# Clean Data
# split into train and test
# Get dimensions, summary, etc. of data